In [ ]:
!unzip fb100.zip

In [13]:
import networkx as nx
import numpy as np
import random
from scipy.sparse import diags
import os
from collections import defaultdict
import pandas as pd
from tqdm import tqdm

def label_propagation(G, labels, max_iter=1000, tol=1e-6):
    """
    Implements label propagation algorithm for recovering missing node attributes

    Parameters:
    -----------
    G : networkx.Graph
        The input graph
    labels : dict
        Dictionary of node:label pairs for known labels
    max_iter : int
        Maximum number of iterations
    tol : float
        Convergence tolerance

    Returns:
    --------
    dict : Predicted labels for all nodes
    """
    node_to_idx = {node: idx for idx, node in enumerate(G.nodes())}
    idx_to_node = {idx: node for node, idx in node_to_idx.items()}

    A = nx.adjacency_matrix(G)
    row_sum = np.array(A.sum(axis=1)).flatten()
    row_sum[row_sum == 0] = 1  # Avoid division by zero
    Dinv = diags(1.0 / row_sum)
    P = Dinv @ A

    n_nodes = len(G.nodes)
    unique_labels = sorted(set(labels.values()))
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    Y = np.zeros((n_nodes, len(unique_labels)))

    for node, label in labels.items():
        if label is not None:
            Y[node_to_idx[node], label_to_index[label]] = 1

    for _ in range(max_iter):
        Y_prev = Y.copy()
        Y = P.dot(Y)

        for node in labels:
            if labels[node] is not None:
                Y[node_to_idx[node], :] = Y_prev[node_to_idx[node], :]

        if np.linalg.norm(Y - Y_prev) < tol:
            break

    index_to_label = {idx: label for label, idx in label_to_index.items()}
    predicted_labels = {}
    for idx in range(n_nodes):
        node = idx_to_node[idx]
        if Y[idx].sum() > 0:
            predicted_labels[node] = index_to_label[np.argmax(Y[idx, :])]

    return predicted_labels

def randomly_remove_attributes(node_attrs, remove_fraction):
    """
    Randomly remove a fraction of node attributes
    """
    known_labels = {node: attr for node, attr in node_attrs.items() if attr is not None}
    total_to_remove = int(len(known_labels) * remove_fraction)
    removed_nodes = random.sample(list(known_labels.keys()), total_to_remove)

    incomplete_attrs = node_attrs.copy()
    for node in removed_nodes:
        incomplete_attrs[node] = None

    # Return both incomplete attributes and remaining known labels
    return incomplete_attrs, {node: attr for node, attr in node_attrs.items()
                            if node not in removed_nodes and attr is not None}

def evaluate_attribute_recovery(G, attribute_name, remove_fractions=[0.1, 0.2, 0.3]):
    """
    Evaluate label propagation for recovering missing attributes
    """
    # Get original attributes
    original_attrs = {node: G.nodes[node].get(attribute_name)
                     for node in G.nodes
                     if G.nodes[node].get(attribute_name) is not None}

    results = defaultdict(dict)

    for fraction in remove_fractions:
        accuracies = []
        for trial in range(3):  
            incomplete_attrs, known_labels = randomly_remove_attributes(original_attrs, fraction)

            predicted_labels = label_propagation(G, known_labels)

            missing_nodes = [node for node in incomplete_attrs if incomplete_attrs[node] is None]
            true_labels = [original_attrs[node] for node in missing_nodes
                          if node in original_attrs and original_attrs[node] is not None]
            recovered_labels = [predicted_labels.get(node) for node in missing_nodes
                              if node in predicted_labels]

            if true_labels and recovered_labels:
                accuracy = sum(1 for y, y_pred in zip(true_labels, recovered_labels)
                             if y == y_pred) / len(true_labels)
                accuracies.append(accuracy)

        results[fraction]['accuracy'] = np.mean(accuracies) if accuracies else 0

    return results

def main():
    data_path = "data"  
    networks_to_analyze = [ 'Carnegie49.gml']
    attributes = ['dorm', 'major_index', 'gender']
    remove_fractions = [0.1, 0.2, 0.3]

    all_results = defaultdict(dict)

    for network_file in networks_to_analyze:
        print(f"\nAnalyzing {network_file}...")
        G = nx.read_gml(os.path.join(data_path, network_file))

        for attr in attributes:
            print(f"\nEvaluating {attr} recovery...")
            results = evaluate_attribute_recovery(G, attr, remove_fractions)

            for fraction, metrics in results.items():
                all_results[network_file][f"{attr}_{fraction}"] = metrics['accuracy']

    results_df = pd.DataFrame(all_results).round(3)
    print("\nResults Summary:")
    print(results_df)

if __name__ == "__main__":
    main()


Analyzing Carnegie49.gml...

Evaluating dorm recovery...

Evaluating major_index recovery...

Evaluating gender recovery...

Results Summary:
                 Carnegie49.gml
dorm_0.1                  0.567
dorm_0.2                  0.498
dorm_0.3                  0.429
major_index_0.1           0.431
major_index_0.2           0.336
major_index_0.3           0.361
gender_0.1                0.582
gender_0.2                0.597
gender_0.3                0.527


In [3]:
import networkx as nx
import numpy as np
import random
from scipy.sparse import diags
from sklearn.metrics import accuracy_score, mean_absolute_error
from tqdm import tqdm

def label_propagation(G, labels, max_iter=1000, tol=1e-6):
    """
    Implements label propagation algorithm for recovering missing node attributes
    """
    node_to_idx = {node: idx for idx, node in enumerate(G.nodes())}
    idx_to_node = {idx: node for node, idx in node_to_idx.items()}

    A = nx.adjacency_matrix(G)
    row_sum = np.array(A.sum(axis=1)).flatten()
    row_sum[row_sum == 0] = 1

    Dinv = diags(1.0 / row_sum)
    P = Dinv @ A

    n_nodes = len(G.nodes)
    unique_labels = sorted(set(labels.values()))
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    Y = np.zeros((n_nodes, len(unique_labels)))

    for node, label in labels.items():
        if label is not None:
            Y[node_to_idx[node], label_to_index[label]] = 1

    for _ in range(max_iter):
        Y_prev = Y.copy()
        Y = P.dot(Y)
        for node in labels:
            if labels[node] is not None:
                Y[node_to_idx[node], :] = Y_prev[node_to_idx[node], :]
        if np.linalg.norm(Y - Y_prev) < tol:
            break

    index_to_label = {idx: label for label, idx in label_to_index.items()}
    predicted_labels = {}
    for idx in range(n_nodes):
        node = idx_to_node[idx]
        if Y[idx].sum() > 0:
            predicted_labels[node] = index_to_label[np.argmax(Y[idx, :])]

    return predicted_labels

def randomly_remove_attributes(node_attrs, remove_fraction):
    """
    Randomly remove a fraction of node attributes
    """
    known_labels = {node: attr for node, attr in node_attrs.items() if attr is not None}
    total_to_remove = int(len(known_labels) * remove_fraction)
    removed_nodes = random.sample(list(known_labels.keys()), total_to_remove)

    incomplete_attrs = node_attrs.copy()
    for node in removed_nodes:
        incomplete_attrs[node] = None

    return incomplete_attrs, {node: attr for node, attr in node_attrs.items()
                            if node not in removed_nodes and attr is not None}

def evaluate_label_propagation(G, attribute_name, remove_fractions=[0.1, 0.2, 0.3]):
    """
    Evaluate label propagation for attribute recovery with multiple metrics
    """
    original_attrs = {node: G.nodes[node].get(attribute_name)
                     for node in G.nodes
                     if G.nodes[node].get(attribute_name) is not None}

    results = {}
    for fraction in remove_fractions:
        print(f"\nEvaluating {attribute_name} with {fraction*100}% removed")

        accuracies = []
        maes = []
        for trial in range(3):
            incomplete_attrs, known_labels = randomly_remove_attributes(original_attrs, fraction)

            predicted_labels = label_propagation(G, known_labels)

            missing_nodes = [node for node in incomplete_attrs if incomplete_attrs[node] is None]
            true_labels = [original_attrs[node] for node in missing_nodes
                         if node in original_attrs and original_attrs[node] is not None]
            recovered_labels = [predicted_labels.get(node) for node in missing_nodes
                             if node in predicted_labels]

            if true_labels and recovered_labels:
                accuracy = sum(1 for y, y_pred in zip(true_labels, recovered_labels)
                             if y == y_pred) / len(true_labels)
                accuracies.append(accuracy)

                try:
                    mae = mean_absolute_error([float(y) for y in true_labels],
                                            [float(y_pred) for y_pred in recovered_labels])
                    maes.append(mae)
                except (ValueError, TypeError):
                    maes.append(None)

        avg_accuracy = np.mean(accuracies) if accuracies else 0
        avg_mae = np.mean([mae for mae in maes if mae is not None]) if maes else None

        results[fraction] = {
            'accuracy': avg_accuracy,
            'mae': avg_mae
        }

    return results

def main():
    print("Loading Carnegie49 network...")
    G = nx.read_gml("data/Carnegie49.gml")

    attributes = ['dorm', 'major_index', 'gender']
    remove_fractions = [0.1, 0.2, 0.3]

    print("\nResults for Carnegie49:")
    print("-" * 50)

    for attr in attributes:
        print(f"\nAttribute: {attr}")
        results = evaluate_label_propagation(G, attr, remove_fractions)

        print("\nFraction | Accuracy | MAE")
        print("-" * 30)
        for fraction, metrics in results.items():
            mae_str = f"{metrics['mae']:.4f}" if metrics['mae'] is not None else "N/A"
            print(f"{fraction*100:>7.1f}% | {metrics['accuracy']:.4f} | {mae_str}")

if __name__ == "__main__":
    main()

Loading Carnegie49 network...

Results for Carnegie49:
--------------------------------------------------

Attribute: dorm

Evaluating dorm with 10.0% removed

Evaluating dorm with 20.0% removed

Evaluating dorm with 30.0% removed

Fraction | Accuracy | MAE
------------------------------
   10.0% | 0.5852 | 26.0402
   20.0% | 0.5566 | 25.9254
   30.0% | 0.4527 | 26.7474

Attribute: major_index

Evaluating major_index with 10.0% removed

Evaluating major_index with 20.0% removed

Evaluating major_index with 30.0% removed

Fraction | Accuracy | MAE
------------------------------
   10.0% | 0.3791 | 25.0739
   20.0% | 0.2195 | 24.5976
   30.0% | 0.3126 | 25.2948

Attribute: gender

Evaluating gender with 10.0% removed

Evaluating gender with 20.0% removed

Evaluating gender with 30.0% removed

Fraction | Accuracy | MAE
------------------------------
   10.0% | 0.6365 | 0.4098
   20.0% | 0.5810 | 0.4408
   30.0% | 0.5451 | 0.4234


# Question 6

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict


class HomophilyAnalyzer:
    def __init__(self, graph_file):
        """Initialize with a GML graph file from FB100 dataset"""
        self.G = nx.read_gml(graph_file)

    def calculate_edge_homophily(self):
        """Calculate proportion of edges between same gender/year nodes"""
        total_edges = self.G.number_of_edges()
        same_gender_edges = 0
        same_year_edges = 0

        for edge in self.G.edges():
            node1, node2 = edge
            if self.G.nodes[node1]['gender'] == self.G.nodes[node2]['gender']:
                same_gender_edges += 1
            if self.G.nodes[node1]['year'] == self.G.nodes[node2]['year']:
                same_year_edges += 1

        return {
            'gender_homophily': same_gender_edges / total_edges,
            'year_homophily': same_year_edges / total_edges
        }

    def calculate_mixing_matrices(self):
        """Calculate mixing matrices for gender and year"""
        gender_mixing = defaultdict(lambda: defaultdict(int))
        year_mixing = defaultdict(lambda: defaultdict(int))

        for edge in self.G.edges():
            node1, node2 = edge
            gender1 = self.G.nodes[node1]['gender']
            gender2 = self.G.nodes[node2]['gender']
            year1 = self.G.nodes[node1]['year']
            year2 = self.G.nodes[node2]['year']

            gender_mixing[gender1][gender2] += 1
            year_mixing[year1][year2] += 1

        return gender_mixing, year_mixing

    def calculate_attribute_assortativity(self):
        """Calculate assortativity coefficients for gender and year"""
        gender_assortativity = nx.attribute_assortativity_coefficient(self.G, 'gender')
        year_assortativity = nx.attribute_assortativity_coefficient(self.G, 'year')
        return gender_assortativity, year_assortativity

    def plot_comparison(self, gender_assort, year_assort, gender_homo, year_homo):
        """Plot comparison of gender vs year influence"""
        labels = ['Assortativity', 'Edge Homophily']
        gender_scores = [gender_assort, gender_homo]
        year_scores = [year_assort, year_homo]

        x = np.arange(len(labels))
        width = 0.35

        fig, ax = plt.subplots(figsize=(10, 6))
        rects1 = ax.bar(x - width/2, gender_scores, width, label='Gender')
        rects2 = ax.bar(x + width/2, year_scores, width, label='Year')

        ax.set_ylabel('Score')
        ax.set_title('Gender vs Year Influence on Network Formation')
        ax.set_xticks(x)
        ax.set_xticklabels(labels)
        ax.legend()

        plt.tight_layout()
        plt.show()

def analyze_multiple_networks(graph_files):
    """Analyze multiple networks and compare results"""
    results = {}

    global gender_homophily_counter, year_homophily_counter, gender_assort_counter, year_assort_counter
    gender_homophily_counter = 0
    year_homophily_counter = 0
    gender_assort_counter = 0
    year_assort_counter = 0


    for graph_file in graph_files:
        print(f"\nAnalyzing {graph_file}...")
        analyzer = HomophilyAnalyzer(graph_file)

        homophily = analyzer.calculate_edge_homophily()
        gender_assort, year_assort = analyzer.calculate_attribute_assortativity()

        results[graph_file] = {
            'gender_homophily': homophily['gender_homophily'],
            'year_homophily': homophily['year_homophily'],
            'gender_assortativity': gender_assort,
            'year_assortativity': year_assort
        }

        # Plot comparison
        ''' analyzer.plot_comparison(
            gender_assort,
            year_assort,
            homophily['gender_homophily'],
            homophily['year_homophily']
        ) '''

        print(f"Gender Homophily: {homophily['gender_homophily']:.3f}")
        print(f"Year Homophily: {homophily['year_homophily']:.3f}")
        print(f"Gender Assortativity: {gender_assort:.3f}")
        print(f"Year Assortativity: {year_assort:.3f}")


        if homophily['gender_homophily']> homophily['year_homophily']:
          gender_homophily_counter +=1
        else:
          year_homophily_counter +=1

        if gender_assort > year_assort:
          gender_assort_counter +=1
        else:
          year_assort_counter +=1

    return results


graph_files = [
    './data/American75.gml',
    './data/Cornell5.gml',
    './data/MU78.gml',
    './data/Rice31.gml',
    './data/UC61.gml',
    './data/USFCA72.gml',
    './data/Amherst41.gml',
    './data/Dartmouth6.gml',
    './data/Maine59.gml',
    './data/Rochester38.gml',
    './data/UC64.gml',
    './data/UVA16.gml',
    './data/Auburn71.gml',
    './data/Duke14.gml',
    './data/Maryland58.gml',
    './data/Rutgers89.gml',
    './data/UCF52.gml',
    './data/Vanderbilt48.gml',
    './data/BC17.gml',
    './data/Emory27.gml',
    './data/Mich67.gml',
    './data/Santa74.gml',
    './data/UCLA26.gml',
    './data/Vassar85.gml',
    './data/BU10.gml',
    './data/FSU53.gml',
    './data/Michigan23.gml',
    './data/Simmons81.gml',
    './data/UCSB37.gml',
    './data/Vermont70.gml',
    './data/Baylor93.gml',
    './data/GWU54.gml',
    './data/Middlebury45.gml',
    './data/Smith60.gml',
    './data/UCSC68.gml',
    './data/Villanova62.gml',
    './data/Berkeley13.gml',
    './data/Georgetown15.gml',
    './data/Mississippi66.gml',
    './data/Stanford3.gml',
    './data/UCSD34.gml',
    './data/Virginia63.gml',
    './data/Bingham82.gml',
    './data/Hamilton46.gml',
    './data/NYU9.gml',
    './data/Swarthmore42.gml',
    './data/UChicago30.gml',
    './data/Wake73.gml',
    './data/Bowdoin47.gml',
    './data/Harvard1.gml',
    './data/Northeastern19.gml',
    './data/Syracuse56.gml',
    './data/UConn91.gml',
    './data/WashU32.gml',
    './data/Brandeis99.gml',
    './data/Haverford76.gml',
    './data/Northwestern25.gml',
    './data/Temple83.gml',
    './data/UF21.gml',
    './data/Wellesley22.gml',
    './data/Brown11.gml',
    './data/Howard90.gml',
    './data/Notre Dame57.gml',
    './data/Tennessee95.gml',
    './data/UGA50.gml',
    './data/Wesleyan43.gml',
    './data/Bucknell39.gml',
    './data/Indiana69.gml',
    './data/Oberlin44.gml',
    './data/Texas80.gml',
    './data/UIllinois20.gml',
    './data/William77.gml',
    './data/Cal65.gml',
    './data/JMU79.gml',
    './data/Oklahoma97.gml',
    './data/Texas84.gml',
    './data/UMass92.gml',
    './data/Williams40.gml',
    './data/Caltech36.gml',
    './data/Johns Hopkins55.gml',
    './data/Penn94.gml',
    './data/Trinity100.gml',
    './data/UNC28.gml',
    './data/Wisconsin87.gml',
    './data/Carnegie49.gml',
    './data/Lehigh96.gml',
    './data/Pepperdine86.gml',
    './data/Tufts18.gml',
    './data/UPenn7.gml',
    './data/Yale4.gml',
    './data/Colgate88.gml',
    './data/MIT8.gml',
    './data/Princeton12.gml',
    './data/Tulane29.gml',
    './data/USC35.gml',
    './data/Columbia2.gml',
    './data/MSU24.gml',
    './data/Reed98.gml',
    './data/UC33.gml',
    './data/USF51.gml',
]


results = analyze_multiple_networks(graph_files)

print("Gender Homophily bigger:", gender_homophily_counter, "out of:", gender_homophily_counter + year_homophily_counter )
print("Gender assor bigger:", gender_assort_counter, "out of:", gender_assort_counter + year_assort_counter )


Analyzing ./data/American75.gml...
Gender Homophily: 0.499
Year Homophily: 0.469
Gender Assortativity: 0.026
Year Assortativity: 0.352

Analyzing ./data/Cornell5.gml...
Gender Homophily: 0.479
Year Homophily: 0.491
Gender Assortativity: 0.072
Year Assortativity: 0.394

Analyzing ./data/MU78.gml...
Gender Homophily: 0.529
Year Homophily: 0.523
Gender Assortativity: 0.102
Year Assortativity: 0.417

Analyzing ./data/Rice31.gml...
Gender Homophily: 0.480
Year Homophily: 0.398
Gender Assortativity: 0.027
Year Assortativity: 0.276

Analyzing ./data/UC61.gml...
Gender Homophily: 0.454
Year Homophily: 0.419
Gender Assortativity: 0.004
Year Assortativity: 0.290

Analyzing ./data/USFCA72.gml...
Gender Homophily: 0.522
Year Homophily: 0.491
Gender Assortativity: 0.022
Year Assortativity: 0.364

Analyzing ./data/Amherst41.gml...
Gender Homophily: 0.465
Year Homophily: 0.513
Gender Assortativity: 0.047
Year Assortativity: 0.409

Analyzing ./data/Dartmouth6.gml...
Gender Homophily: 0.468
Year Homop

KeyboardInterrupt: 

In [2]:

print("Gender Homophily bigger:", gender_homophily_counter, "out of:", gender_homophily_counter + year_homophily_counter )
print("Gender assor bigger:", gender_assort_counter, "out of:", gender_assort_counter + year_assort_counter )

Gender Homophily bigger: 12 out of: 22
Gender assor bigger: 0 out of: 22
